In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from Config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lebu,CL,-37.6167,-73.6500,47.71,80,3,10.09,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,38.86,70,0,0.00,clear sky
2,2,Tura,IN,25.5198,90.2201,76.53,92,34,3.47,scattered clouds
3,3,Lakes Entrance,AU,-37.8811,147.9810,67.15,41,97,10.11,overcast clouds
4,4,Andenes,NO,69.3143,16.1194,48.18,81,90,10.36,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Tura,IN,25.5198,90.2201,76.53,92,34,3.47,scattered clouds
8,8,Celestun,MX,20.8667,-90.4000,81.90,84,65,13.89,broken clouds
10,10,Tomohon,ID,1.3346,124.8392,79.74,69,83,2.37,broken clouds
14,14,Atuona,PF,-9.8000,-139.0333,77.92,74,18,17.40,few clouds
19,19,Port Blair,IN,11.6667,92.7500,82.44,89,48,5.75,scattered clouds
21,21,Cartagena,CO,10.3997,-75.5144,81.75,83,32,12.55,scattered clouds
24,24,Nautla,MX,20.2167,-96.7833,77.76,85,100,6.17,overcast clouds
25,25,Kerema,PG,-7.9631,145.7785,79.79,87,75,5.19,moderate rain
30,30,Tera,NE,14.0078,0.7531,84.02,57,21,11.41,few clouds
32,32,Hilo,US,19.7297,-155.0900,79.25,78,22,3.00,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                285
City                   285
Country                285
Lat                    285
Lng                    285
Max Temp               285
Humidity               285
Cloudiness             285
Wind Speed             285
Current Description    285
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Tura,IN,76.53,scattered clouds,25.5198,90.2201,
8,Celestun,MX,81.90,broken clouds,20.8667,-90.4000,
10,Tomohon,ID,79.74,broken clouds,1.3346,124.8392,
14,Atuona,PF,77.92,few clouds,-9.8000,-139.0333,
19,Port Blair,IN,82.44,scattered clouds,11.6667,92.7500,
21,Cartagena,CO,81.75,scattered clouds,10.3997,-75.5144,
24,Nautla,MX,77.76,overcast clouds,20.2167,-96.7833,
25,Kerema,PG,79.79,moderate rain,-7.9631,145.7785,
30,Tera,NE,84.02,few clouds,14.0078,0.7531,
32,Hilo,US,79.25,few clouds,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Tura,IN,76.53,scattered clouds,25.5198,90.2201,Hotel RIKMAN Continental
8,Celestun,MX,81.90,broken clouds,20.8667,-90.4000,Posada Lilia
10,Tomohon,ID,79.74,broken clouds,1.3346,124.8392,Gardenia Country Inn
14,Atuona,PF,77.92,few clouds,-9.8000,-139.0333,Villa Enata
19,Port Blair,IN,82.44,scattered clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
21,Cartagena,CO,81.75,scattered clouds,10.3997,-75.5144,Hotel Colombia Real - Cartagena
24,Nautla,MX,77.76,overcast clouds,20.2167,-96.7833,Casa Blanca
25,Kerema,PG,79.79,moderate rain,-7.9631,145.7785,Frena Lodge
30,Tera,NE,84.02,few clouds,14.0078,0.7531,Hotel Dar es salam
32,Hilo,US,79.25,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [36]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure.
fig

Figure(layout=FigureLayout(height='420px'))